<a href="https://colab.research.google.com/github/tncourtz/RL-EV-CS-Energy-Management-System/blob/main/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving trading_data.xlsx to trading_data.xlsx


In [ ]:
from pandas import read_excel
df = read_excel('trading_data.xlsx')

PVt= df['pv'].values
PLt = df['load'].values
GTt = df['gt'].values

# **24-hour PSO**

In [ ]:
from __future__ import division
import random
import numpy as np 
from math import exp
import matplotlib.pyplot as plt
import math


W = 2
c1 = 0.5
c2 = 0.5

n_iterations = int(input("Inform the number of iterations: "))     #number of iterations
n_particles = int(input("Inform the number of particles: "))       #number of iterations

class Particle():
    def __init__(self, n_particles, first):
      if first == 1:
        self.position = np.array(0.5)
      else:
        self.position = np.random.uniform(low=0.1, high=0.9, size=1)       #initialize a soc particle between constraints

      self.pbest_position = self.position.copy()                          #initialize best particle value
      self.pbest_value = float('inf')                                     # intialize the best value to a large number
      self.velocity = np.array(0)                                         #initialize the velocity of the particle
      self.PG = 0
      self.PB = 0 

    #print postion
    def __str__(self):
        print("Current Position", self.position, " Best Position ", self.pbest_position)
    #use velocity to move the particle
    def move(self):
        self.position = self.position + self.velocity
              # adjust maximum position if necessary
        if self.position > 0.9:
          self.position = 0.9

      # adjust minimum position if neseccary
        if self.position < 0.1:
          self.position = 0.1

###############################################################################################################################
class Space():
    def __init__(self, n_particles):
        self.n_particles = n_particles
        self.particles = []
        self.gbest_value = float('inf')                                       #best global value
        self.gbest_position = 0.5      #best global position
        self.Cost = []
        self.is_Best = False
        self.plchld = Particle(1,1)

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()
    
    # cost function
    def fitness(self, particle, GT, PV, PL): #GT,PV, PL are from arrays of the 24h grid prices, pv, and load
        #Variables 
        plch = self.plchld
        Q = 100                               # (kwh) battery full capacity
        Tamb = 23                            #ambient temperature
        Rth = 0.0002                          # battery thermal resistance
        # model parameters
        a = 1.434
        b = 48.808
        m = 3.185
        d = 0.0768
        Yhr = 8760                           # hours per year
        Cbat = 200                           # initial battery cost 

        SOCmin = 0.1
        SOCmax = 0.9
        SOCinit = 0.5
        Pbmin = -18
        Pbmax = 18
        Pgmin = -24
        Pgmax = 24

        #SYSTEM DYNAMICS
        dt =1
        Pb = (-particle.position+plch.position)*Q                          # battery power #-plch.position
        if Pb < -75:
          Pb = -75
        if Pb > 75:
          Pb = 75

        Pg = PL - PV - Pb                                 #grid suppled power
        if Pg < -100:
          Pg = -100
        if Pg > 150:
          Pg = 150

        T = Tamb + Rth*abs(Pb)*1000                            # battery temperature 
        L = a*exp(b/T)                                # battery lifetime 
        SOCavg = particle.position                        # average SOC ?what is the avg. soc
        DOD = particle.position                           # battery depth of discharge

        #COST FUNCTIONS
        Cg = GT*Pg*dt                                     # grid running cost
        Ctemp = Cbat*dt/(Yhr*L)                           # Temperature operation cost
        Csoc = (Cbat*m*SOCavg-d)/(15*0.2*Yhr)             # Cost due to avgSOC 
        Cdod = (-0.012*DOD**4) + (0.033*DOD**3) + (0.021*DOD**2) - (0.056*DOD) # cost due to dod
        Cbd = max(Ctemp, Cdod, Csoc)                      # total battery degradation cost 
        total = Cg + Pb*dt*Cbd
        self.plchld = particle
        return    total, Pg, Pb                          # total system cost

    # find the best position for the particles
    def set_pbest(self ,particle, GT, PV, PL):
        # for particle in self.particles:
            # i = self.particles.index(particle)              # indexing 
        is_Best = False
        fitness_cand, pg, pb = self.fitness(particle, GT, PV, PL)
        if(particle.pbest_value > fitness_cand):
            particle.pbest_value = fitness_cand
            particle.pbest_position = particle.position
            particle.PG = pg
            particle.PB = pb
            is_Best = True
        return is_Best
            
    # find best global position for particles
    def set_gbest(self ,particle, GT, PV, PL):
        # for particle in self.particles:
            # i = self.particles.index(particle)
        best_fitness_cand, pg, pb = self.fitness(particle, GT, PV, PL)
        if(self.gbest_value > best_fitness_cand):
            self.gbest_value = best_fitness_cand
            self.gbest_position = particle.position
    
    #move particles
    def move_particles(self, particle):
        # for particle in self.particles:
      global W
      new_velocity = (W*particle.velocity) + (c1*random.random()) * (particle.pbest_position - particle.position) + \
                      (random.random()*c2) * (self.gbest_position - particle.position)
      particle.velocity = new_velocity
      particle.move()
      return particle

#############################################################################################################################
#IMPLEMENTATION OF PSO ALGO.
iteration = 0
sss = np.zeros(24)
pg = np.zeros(24)
pb = np.zeros(24)
the = np.zeros(24)

while(iteration < n_iterations):
  # if iteration == 0:
  search_space = Space(n_particles)
  particles_vector = np.concatenate(([Particle(1, 1)], [Particle(n_particles-1, 0) for i in range(search_space.n_particles-1)]), axis=None)
  search_space.particles = particles_vector
  the = particles_vector
  search_space.print_particles()
  # else: 
  #   search_space.particles = the
  #   search_space.print_particles()

  for i in range(24):
    part = search_space.particles[i]
    # search_space.set_pbest(part,GTt[i], PVt[i], PLt[i]) 
    is_best = search_space.set_pbest(part,GTt[i], PVt[i], PLt[i])
    search_space.set_gbest(part, GTt[i], PVt[i], PLt[i])
    search_space.move_particles(part)
    yy = search_space.move_particles(part)
    sss[i] = (part.pbest_position)
    pg[i] = part.PG
    pb[i] = part.PB
    # print(is_best)
    # if is_best:
    #   # print(the)
    #   the[i] = yy
  iteration += 1
    
print("The best solution is: ", sss, " in n_iterations: ", iteration)
print(pg)
print(pb)

# **Plots and Calculations**

In [ ]:
tt = sss
soc = []
for i in range(len(tt)):
  soc.append(tt[i])
print(soc)
t = np.arange(0, len(tt))
plt.plot(t, tt)

In [ ]:
tt2 = pg
g = []
for i in range(len(tt2)):
  g.append(tt2[i])
print(g)
t = np.arange(0, len(tt2))
plt.plot(t, tt2)

In [ ]:
tt = pb
b = []
for i in range(len(tt)):
  b.append(tt[i])
print(b)
t = np.arange(0, len(tt))
plt.plot(t, tt)

In [ ]:

L=0
f=[]
for i in range(24):
  # L += Pb[i]
  T = 23+ 0.0002*abs(pb[i])*1000
  L += 1.434*exp(48.808/T)
L = L/24
print(L)

8.926996338131522


In [ ]:
cost = 0
for i in range(len(pg)):
  cost += GTt[i]*pg[i]

print(cost)

86.71272971238976
